# 微信聊天角色扮演 - Qwen3-0.6B 微调

使用LoRA微调Qwen3-0.6B，让模型学习模仿聊天记录中对方的说话风格。

## 环境要求
- Google Colab (GPU)
- 免费版T4 GPU即可运行

In [37]:
# 安装依赖
!pip install -q transformers>=4.40.0 peft>=0.10.0 trl>=0.8.0 datasets accelerate bitsandbytes

In [38]:
import json
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from datasets import Dataset

# 检查GPU
print(f"GPU可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU型号: {torch.cuda.get_device_name(0)}")
    print(f"显存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

GPU可用: True
GPU型号: Tesla T4
显存: 15.8 GB


## 1. 上传训练数据

将预处理好的 `train_data.json` 上传到Colab

In [39]:
from google.colab import files

# 上传训练数据
uploaded = files.upload()
data_file = list(uploaded.keys())[0]
print(f"已上传: {data_file}")

Saving train_data.json to train_data (2).json
已上传: train_data (2).json


In [40]:
# 加载训练数据
with open(data_file, 'r', encoding='utf-8') as f:
    train_data = json.load(f)

print(f"训练样本数量: {len(train_data)}")
print(f"\n样本示例:")
print(json.dumps(train_data[0], ensure_ascii=False, indent=2))

训练样本数量: 9

样本示例:
{
  "conversations": [
    {
      "role": "user",
      "content": "到家了没"
    },
    {
      "role": "assistant",
      "content": "今天蟹蟹啦\n把钱转给你\n到啦到啦\n你们到家了吗"
    },
    {
      "role": "user",
      "content": "我们在外面消消食"
    },
    {
      "role": "assistant",
      "content": "好滴好滴，那下回我请"
    }
  ]
}


## 2. 加载模型和Tokenizer

In [41]:
MODEL_ID = "Qwen/Qwen3-0.6B"

# 加载Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True
)

# 设置pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"词表大小: {len(tokenizer)}")
print(f"特殊token: pad={tokenizer.pad_token}, eos={tokenizer.eos_token}")

词表大小: 151669
特殊token: pad=<|endoftext|>, eos=<|im_end|>


In [42]:
# 4-bit量化配置（节省显存）
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# 准备模型进行k-bit训练
model = prepare_model_for_kbit_training(model)

print(f"模型加载完成")
print(f"模型参数量: {model.num_parameters() / 1e6:.1f}M")

Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


模型加载完成
模型参数量: 751.6M


## 3. 配置LoRA

In [43]:
# LoRA配置
lora_config = LoraConfig(
    r=16,                    # LoRA秩
    lora_alpha=32,           # 缩放系数
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # 注意力层
        "gate_proj", "up_proj", "down_proj"       # FFN层
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 应用LoRA
model = get_peft_model(model, lora_config)

# 打印可训练参数
model.print_trainable_parameters()

trainable params: 10,092,544 || all params: 761,724,928 || trainable%: 1.3250


## 4. 准备数据集

In [44]:
def format_conversation(example):
    """
    将对话格式化为Qwen3的chat模板
    """
    messages = example["conversations"]

    # 使用tokenizer的chat模板
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )

    return {"text": text}

# 创建Dataset
dataset = Dataset.from_list(train_data)
dataset = dataset.map(format_conversation)

print(f"数据集大小: {len(dataset)}")
print(f"\n格式化后的样本:")
print(dataset[0]["text"][:500])

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

数据集大小: 9

格式化后的样本:
<|im_start|>user
到家了没<|im_end|>
<|im_start|>assistant
今天蟹蟹啦
把钱转给你
到啦到啦
你们到家了吗<|im_end|>
<|im_start|>user
我们在外面消消食<|im_end|>
<|im_start|>assistant
<think>

</think>

好滴好滴，那下回我请<|im_end|>



## 5. 训练配置

In [80]:
# 训练参数
training_args = SFTConfig(
    output_dir="./output",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    fp16=False,
    logging_steps=1,
    save_strategy="epoch",
    save_total_limit=2,
    optim="paged_adamw_8bit",
    report_to="none",
    seed=42,
    dataset_text_field="text",
    packing=False
)

# 通过tokenizer控制序列长度
tokenizer.model_max_length = 512

print("训练配置:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


训练配置:
  Epochs: 2
  Batch size: 4
  Gradient accumulation: 4
  Effective batch size: 16
  Learning rate: 0.0002


In [81]:
# 创建Trainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer
)

print("Trainer已创建，准备开始训练")

Adding EOS to train dataset:   0%|          | 0/9 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/9 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/9 [00:00<?, ? examples/s]

Trainer已创建，准备开始训练


## 6. 开始训练

In [82]:
# 开始训练
print("开始训练...")
trainer.train()
print("训练完成！")

开始训练...


Step,Training Loss
1,0.022376
2,0.021882


训练完成！


## 7. 保存模型

In [83]:
# 保存LoRA适配器
LORA_OUTPUT_DIR = "./lora_adapter"
model.save_pretrained(LORA_OUTPUT_DIR)
tokenizer.save_pretrained(LORA_OUTPUT_DIR)

print(f"LoRA适配器已保存至: {LORA_OUTPUT_DIR}")

LoRA适配器已保存至: ./lora_adapter


In [84]:
# （可选）合并为完整模型
MERGE_OUTPUT_DIR = "./merged_model"

# 重新加载基座模型（不量化）
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 加载LoRA权重并合并
from peft import PeftModel
merged_model = PeftModel.from_pretrained(base_model, LORA_OUTPUT_DIR)
merged_model = merged_model.merge_and_unload()

# 保存合并后的模型
merged_model.save_pretrained(MERGE_OUTPUT_DIR)
tokenizer.save_pretrained(MERGE_OUTPUT_DIR)

print(f"合并后的模型已保存至: {MERGE_OUTPUT_DIR}")

Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

合并后的模型已保存至: ./merged_model


## 8. 测试模型

In [85]:
def chat(model, tokenizer, user_input, history=[]):
    """
    单轮对话测试
    """
    messages = history + [{"role": "user", "content": user_input}]

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            pad_token_id=tokenizer.pad_token_id
        )

    generated = outputs[0][inputs["input_ids"].shape[1]:]
    response = tokenizer.decode(generated, skip_special_tokens=True)

    return response.strip()

In [86]:
# 测试对话
test_inputs = [
    "在干嘛",
    "今天累不累",
    "晚上吃什么",
    "一起看电影吧"
]

print("=== 模型测试 ===")
for user_input in test_inputs:
    response = chat(merged_model, tokenizer, user_input)
    print(f"用户: {user_input}")
    print(f"模型: {response}")
    print()

=== 模型测试 ===
用户: 在干嘛
模型: 回个声不
你在看小说吗

用户: 今天累不累
模型: <think>

</think>

不好意思，我感觉有点累

用户: 晚上吃什么
模型: <think>

</think>

那可以先去便利店买点东西不

用户: 一起看电影吧
模型: 好滴，那我可以问问你们几点开始呀



## 9. 下载模型

将训练好的模型下载到本地

In [87]:
# 打包LoRA适配器
!zip -r lora_adapter.zip ./lora_adapter

# 下载
from google.colab import files
files.download('lora_adapter.zip')

print("LoRA适配器已下载，解压后放到本地项目目录使用")

updating: lora_adapter/ (stored 0%)
updating: lora_adapter/adapter_config.json (deflated 59%)
updating: lora_adapter/README.md (deflated 65%)
updating: lora_adapter/chat_template.jinja (deflated 76%)
updating: lora_adapter/tokenizer_config.json (deflated 59%)
updating: lora_adapter/adapter_model.safetensors (deflated 21%)
updating: lora_adapter/tokenizer.json (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

LoRA适配器已下载，解压后放到本地项目目录使用


In [88]:
# （可选）打包合并后的完整模型
# 注意：完整模型文件较大，下载可能需要较长时间
!zip -r merged_model.zip ./merged_model

from google.colab import files
files.download('merged_model.zip')

updating: merged_model/ (stored 0%)
updating: merged_model/chat_template.jinja (deflated 76%)
updating: merged_model/config.json (deflated 71%)
updating: merged_model/model.safetensors (deflated 14%)
updating: merged_model/tokenizer_config.json (deflated 59%)
updating: merged_model/generation_config.json (deflated 38%)
updating: merged_model/tokenizer.json (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 本地使用方法

```bash
# 方法1: 使用LoRA适配器
python inference/chat.py --model Qwen/Qwen3-0.6B --lora ./lora_adapter

# 方法2: 使用合并后的模型
python inference/chat.py --model ./merged_model
```